In [2]:
import numpy as np
import pandas as pd

# Task 0
Read the dataset from csv file & perform data cleaning - remove all rows, which contains `?` in some columns.
Also check for data correctness (salary & salary $K).

In [53]:
df = pd.read_csv("../data/adult.csv", index_col=[0])
df = df.replace("?", np.nan)
df["salary K$"] = pd.to_numeric(df["salary K$"], errors="coerce")
df = df.dropna()
df["salary"].value_counts()

salary
<=50K    22654
>50K      7508
Name: count, dtype: int64

# Task 1
Print the count of men and women in the dataset.

In [56]:
df["sex"].value_counts()

sex
Male      20380
Female     9782
Name: count, dtype: int64

# Task 2
Find the average age of men in dataset

In [59]:
df[df["sex"] == "Male"]["age"].mean()

39.18400392541707

# Task 3
Get the percentage of people from Poland (native-country)

In [70]:
(df["native-country"] == "Poland").mean() * 100

0.18566408063125786

# Task 4
Get the mean and standard deviation of the age for people who earn > 50K per year. After this, get it for those who earn <= 50K.

In [76]:
more_then = df[df["salary"] == ">50K"]["age"]
print(f"For people with salary more then 50k mean is {more_then.mean()}, standard deviation is {more_then.std()}")
less_then = df[df["salary"] == "<=50K"]["age"]
print(f"For people with salary less then or equal 50K mean is {less_then.mean()}, standard deviation is {less_then.std()}")

For people with salary more then 50k mean is 43.95911028236548, standard deviation is 10.269632835673852
For people with salary less then or equal 50K mean is 36.60806038668668, standard deviation is 13.464631257161633


# Task 5
Check, if there are some people without higher education (education: Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters, Doctorate), but with > 50K salary

In [78]:
df[(~df["education"].isin(["Bachelors", "Prof-school", "Assoc-acdm", "Assoc-voc", "Masters", "Doctorate"])) & (df["salary"] == ">50K")]

,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,salary,salary K$
7,52,Self-emp-not-inc,HS-grad,Married-civ-spouse,Exec-managerial,Husband,White,Male,45,United-States,>50K,307
10,37,Private,Some-college,Married-civ-spouse,Exec-managerial,Husband,Black,Male,80,United-States,>50K,116
55,43,Private,Some-college,Married-civ-spouse,Tech-support,Husband,White,Male,40,United-States,>50K,341
67,53,Private,HS-grad,Married-civ-spouse,Adm-clerical,Wife,White,Female,40,United-States,>50K,225
68,49,Self-emp-inc,Some-college,Married-civ-spouse,Exec-managerial,Husband,White,Male,50,United-States,>50K,194
...,...,...,...,...,...,...,...,...,...,...,...,...
32462,48,Self-emp-inc,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,50,United-States,>50K,343
32518,57,Local-gov,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,40,United-States,>50K,116
32519,46,Private,Some-college,Married-civ-spouse,Exec-managerial,Husband,White,Male,48,United-States,>50K,239
32557,40,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,40,United-States,>50K,173


# Task 6
Get the statistics of age for each type of education. Use `groupby` and `describe` for this.

In [79]:
df.groupby("education")["age"].describe()

,count,mean,std,min,25%,50%,75%,max
education,,,,,,,,
10th,820.0,37.897561,16.225795,17.0,23.0,36.0,52.0,90.0
11th,1048.0,32.363550,15.089307,17.0,18.0,28.5,43.0,90.0
12th,377.0,32.013263,14.373710,17.0,19.0,28.0,41.0,79.0
1st-4th,151.0,44.622517,14.929051,19.0,33.0,44.0,56.0,81.0
5th-6th,288.0,41.649306,14.754622,17.0,28.0,41.0,53.0,82.0
7th-8th,557.0,47.631957,15.737479,17.0,34.0,49.0,60.0,90.0
9th,455.0,40.303297,15.335754,17.0,28.0,38.0,53.0,90.0
Assoc-acdm,1008.0,37.286706,10.509755,19.0,29.0,36.0,44.0,90.0
Assoc-voc,1307.0,38.246366,11.181253,19.0,30.0,37.0,45.0,84.0


# Task 7
Compare the married and non-married men salaries. Who earns more? (>50K or <=50K)
Married men are those, whom `marital-status` starts with "Married". Others are not.

In [81]:
married_men = df[(df["sex"] == "Male") & (df["marital-status"].str.startswith("Married"))]
non_married_men = df[(df["sex"] == "Male") & ~(df["marital-status"].str.startswith("Married"))]

married_more_than_50k = (married_men["salary"] == ">50K").mean() * 100
non_married_more_than_50k = (non_married_men["salary"] == ">50K").mean() * 100
married_more_than_50k > non_married_more_than_50k

True

# Task 8
Get the max hours per week some person works. How many people works the same amount of hours per week?

In [83]:
max_hours_per_week = df["hours-per-week"].max()

people_with_max_hours = df[df["hours-per-week"] == max_hours_per_week]
len(people_with_max_hours)

78

# Task 9
Analyze the correlation between data in dataset. Understand connected fields in it and print highlight their connection.

In [93]:
df[["age", "hours-per-week", "salary K$"]].corr()

,age,hours-per-week,salary K$
age,1.000000,0.101599,0.208203
hours-per-week,0.101599,1.000000,0.196378
salary K$,0.208203,0.196378,1.000000
